In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
from sklearn.preprocessing import LabelEncoder
from statsmodels.tsa.stattools import grangercausalitytests

data = pd.read_csv('/content/drive/MyDrive/FYP Data/Main/Mississippi River CSV.csv')
data = data.drop(['Year', 'Month', 'Day'], axis=1)

data

,Date,Min Temperature,Max Temperature,Precipitation,Min_Rel Humidity,Max_Rel Humidity,Wind Speed,Vapor Pressure Deficit,Flood
0,1/1/1979,-3.75,11.75,3.8,64.3,98.7,6.2,0.29,False
1,1/2/1979,-8.45,-0.65,0.0,24.5,87.6,6.3,0.26,False
2,1/3/1979,-8.25,3.85,0.0,29.0,76.8,1.5,0.34,False
3,1/4/1979,-5.55,7.85,0.7,29.7,58.8,3.1,0.41,False
4,1/5/1979,-0.15,7.95,13.6,39.0,100.0,3.1,0.22,False
...,...,...,...,...,...,...,...,...,...
16063,12/24/2022,-8.05,0.55,0.0,17.2,36.2,4.0,0.36,False
16064,12/25/2022,-6.65,5.15,0.0,21.0,52.8,1.7,0.42,False
16065,12/26/2022,-4.85,10.45,0.0,19.6,62.8,2.5,0.56,False
16066,12/27/2022,-2.95,11.25,0.0,48.3,100.0,3.1,0.26,False


In [ ]:
data.index = pd.to_datetime(data['Date'], format='%m/%d/%Y')
data = data.drop(['Date'], axis=1)

le = LabelEncoder()
data['Flood'] = le.fit_transform(data.Flood.values)

data

,Min Temperature,Max Temperature,Precipitation,Min_Rel Humidity,Max_Rel Humidity,Wind Speed,Vapor Pressure Deficit,Flood
Date,,,,,,,,
1979-01-01,-3.75,11.75,3.8,64.3,98.7,6.2,0.29,0
1979-01-02,-8.45,-0.65,0.0,24.5,87.6,6.3,0.26,0
1979-01-03,-8.25,3.85,0.0,29.0,76.8,1.5,0.34,0
1979-01-04,-5.55,7.85,0.7,29.7,58.8,3.1,0.41,0
1979-01-05,-0.15,7.95,13.6,39.0,100.0,3.1,0.22,0
...,...,...,...,...,...,...,...,...
2022-12-24,-8.05,0.55,0.0,17.2,36.2,4.0,0.36,0
2022-12-25,-6.65,5.15,0.0,21.0,52.8,1.7,0.42,0
2022-12-26,-4.85,10.45,0.0,19.6,62.8,2.5,0.56,0


In [ ]:
maxlag=30

test = 'ssr_chi2test'
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

grangers_causation_matrix(data, variables = data.columns)

/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 2, but rank is 1
  warnings.warn('covariance of constraints does not have full '


,Min Temperature_x,Max Temperature_x,Precipitation_x,Min_Rel Humidity_x,Max_Rel Humidity_x,Wind Speed_x,Vapor Pressure Deficit_x,Flood_x
Min Temperature_y,1.0000,0.0000,0.0,0.000,0.0000,0.0000,0.0000,0.0013
Max Temperature_y,0.0000,1.0000,0.0,0.000,0.0000,0.0000,0.0000,0.0376
Precipitation_y,0.0000,0.0000,1.0,0.000,0.0000,0.0000,0.0001,0.0228
Min_Rel Humidity_y,0.0000,0.0000,0.0,1.000,0.0000,0.0000,0.0000,0.1440
Max_Rel Humidity_y,0.0000,0.0000,0.0,0.000,1.0000,0.0000,0.0000,0.0000
Wind Speed_y,0.0000,0.0000,0.0,0.000,0.0000,1.0000,0.0000,0.0001
Vapor Pressure Deficit_y,0.0000,0.0000,0.0,0.000,0.0000,0.0000,1.0000,0.0039
Flood_y,0.1541,0.0383,0.0,0.055,0.0005,0.0002,0.0220,1.0000


In [ ]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

def cointegration_test(data, alpha=0.05): 
    out = coint_johansen(data,-1,5)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-alpha)]]
    def adjust(val, length= 6): return str(val).ljust(length)

    print('Name   ::  Test Stat > C(95%)    =>   Signif  \n', '--'*20)
    for col, trace, cvt in zip(data.columns, traces, cvts):
        print(adjust(col), ':: ', adjust(round(trace,2), 9), ">", adjust(cvt, 8), ' =>  ' , trace > cvt)

cointegration_test(data)

Name   ::  Test Stat > C(95%)    =>   Signif  
 ----------------------------------------
Min Temperature ::  7889.3    > 143.6691  =>   True
Max Temperature ::  5419.82   > 111.7797  =>   True
Precipitation ::  3415.32   > 83.9383   =>   True
Min_Rel Humidity ::  2043.39   > 60.0627   =>   True
Max_Rel Humidity ::  989.5     > 40.1749   =>   True
Wind Speed ::  337.33    > 24.2761   =>   True
Vapor Pressure Deficit ::  130.38    > 12.3212   =>   True
Flood  ::  2.44      > 4.1296    =>   False


In [ ]:
nobs = 4000
df_train, df_test = data[0:-nobs], data[-nobs:]

In [ ]:
def adfuller_test(series, signif=0.05, name='', verbose=False):
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")

for name, column in df_train.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "Min Temperature" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -7.6081
 No. Lags Chosen       = 40
 Critical value 1%     = -3.431
 Critical value 5%     = -2.862
 Critical value 10%    = -2.567
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "Max Temperature" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -7.7665
 No. Lags Chosen       = 40
 Critical value 1%     = -3.431
 Critical value 5%     = -2.862
 Critical value 10%    = -2.567
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "Precipitation" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationar

In [ ]:
model = VAR(df_train)

x = model.select_order(maxlags=30)
x.summary()

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'


,AIC,BIC,FPE,HQIC
0,12.93,12.93,4.121e+05,12.93
1,5.252,5.297,191.0,5.267
2,4.991,5.075,147.1,5.019
3,4.911,5.034,135.8,4.953
4,4.867,5.029*,129.9,4.921
5,4.836,5.038,126.0,4.904
6,4.818,5.059,123.8,4.899
7,4.800,5.080,121.5,4.894*
8,4.792,5.111,120.5,4.899
9,4.788,5.147,120.1,4.909


In [ ]:
model_fitted = model.fit(7)
model_fitted.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Wed, 04, Jan, 2023
Time:                     03:54:29
--------------------------------------------------------------------
No. of Equations:         8.00000    BIC:                    5.10000
Nobs:                     12061.0    HQIC:                   4.91411
Log likelihood:          -165523.    FPE:                    124.004
AIC:                      4.82031    Det(Omega_mle):         119.414
--------------------------------------------------------------------
Results for equation Min Temperature
                               coefficient       std. error           t-stat            prob
--------------------------------------------------------------------------------------------
const                             0.837814         0.659245            1.271           0.204
L1.Min Temperature                0.600219         0.012061           49.766           0.000
L1

In [ ]:
from statsmodels.stats.stattools import durbin_watson
out = durbin_watson(model_fitted.resid)

for col, val in zip(data.columns, out):
    print((col), ':', round(val, 2))

Min Temperature : 2.01
Max Temperature : 2.01
Precipitation : 2.0
Min_Rel Humidity : 2.0
Max_Rel Humidity : 2.01
Wind Speed : 2.01
Vapor Pressure Deficit : 2.01
Flood : 2.0


In [ ]:
lag_order = model_fitted.k_ar
print(lag_order)

forecast_input = df_train.values[-lag_order:]
forecast_input

7


array([[ 15.25,  23.15,   8.4 ,  56.6 ,  87.6 ,   2.7 ,   0.68,   0.  ],
       [ 14.35,  22.85,   7.4 ,  56.1 ,  90.4 ,   3.6 ,   0.65,   0.  ],
       [  8.05,  16.45,   0.  ,  62.5 , 100.  ,   5.9 ,   0.31,   0.  ],
       [ -2.65,   9.55,   0.  ,  60.3 , 100.  ,   5.9 ,   0.13,   0.  ],
       [ -3.55,   8.25,   0.  ,  28.2 ,  64.9 ,   3.7 ,   0.46,   0.  ],
       [ -3.05,  16.05,   0.  ,  19.5 ,  75.8 ,   4.7 ,   0.76,   0.  ],
       [ -1.35,  17.55,   0.  ,  26.9 ,  97.3 ,   3.1 ,   0.71,   0.  ]])

In [ ]:
fc = model_fitted.forecast(y=forecast_input, steps=nobs)
df_forecast = pd.DataFrame(fc, index=data.index[-nobs:], columns=data.columns + '_2d')
df_forecast

,Min Temperature_2d,Max Temperature_2d,Precipitation_2d,Min_Rel Humidity_2d,Max_Rel Humidity_2d,Wind Speed_2d,Vapor Pressure Deficit_2d,Flood_2d
Date,,,,,,,,
2012-01-16,1.844034,18.343397,-0.685768,35.975243,87.469908,3.310527,0.735747,0.000960
2012-01-17,4.759930,19.409054,4.732697,40.354772,88.227198,3.976552,0.723297,-0.002921
2012-01-18,5.146723,18.715349,5.012404,42.035813,89.435965,4.301276,0.673013,-0.005252
2012-01-19,4.407076,17.792931,5.278955,41.762321,90.110071,4.404887,0.620911,-0.005586
2012-01-20,3.822815,17.247585,4.396389,39.422928,88.931086,4.203058,0.635828,-0.004599
...,...,...,...,...,...,...,...,...
2022-12-24,12.091960,24.791361,4.017341,44.415646,92.966964,3.470220,0.876475,0.023609
2022-12-25,12.091960,24.791361,4.017341,44.415646,92.966964,3.470220,0.876475,0.023609
2022-12-26,12.091960,24.791361,4.017341,44.415646,92.966964,3.470220,0.876475,0.023609


In [ ]:
from statsmodels.tsa.stattools import acf
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    mins = np.amin(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    maxs = np.amax(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse, 'corr':corr, 'minmax':minmax})

print('Forecast Accuracy of: Min Temperature')
accuracy_prod = forecast_accuracy(df_forecast['Min Temperature_2d'].values, df_test['Min Temperature'])
for k, v in accuracy_prod.items():
    print((k), ': ', round(v,4))

print('\nForecast Accuracy of: Max Temperature')
accuracy_prod = forecast_accuracy(df_forecast['Max Temperature_2d'].values, df_test['Max Temperature'])
for k, v in accuracy_prod.items():
    print((k), ': ', round(v,4))

print('\nForecast Accuracy of: Precipitation')
accuracy_prod = forecast_accuracy(df_forecast['Precipitation_2d'].values, df_test['Precipitation'])
for k, v in accuracy_prod.items():
    print((k), ': ', round(v,4))

print('\nForecast Accuracy of: Min_Rel Humidity')
accuracy_prod = forecast_accuracy(df_forecast['Min_Rel Humidity_2d'].values, df_test['Min_Rel Humidity'])
for k, v in accuracy_prod.items():
    print((k), ': ', round(v,4))

print('\nForecast Accuracy of: Max_Rel Humidity')
accuracy_prod = forecast_accuracy(df_forecast['Max_Rel Humidity_2d'].values, df_test['Max_Rel Humidity'])
for k, v in accuracy_prod.items():
    print((k), ': ', round(v,4))

print('\nForecast Accuracy of: Wind Speed')
accuracy_prod = forecast_accuracy(df_forecast['Wind Speed_2d'].values, df_test['Wind Speed'])
for k, v in accuracy_prod.items():
    print((k), ': ', round(v,4))

print('\nForecast Accuracy of: Vapor Pressure Deficit')
accuracy_prod = forecast_accuracy(df_forecast['Vapor Pressure Deficit_2d'].values, df_test['Vapor Pressure Deficit'])
for k, v in accuracy_prod.items():
    print((k), ': ', round(v,4))

print('\nForecast Accuracy of: Flood')
accuracy_prod = forecast_accuracy(df_forecast['Flood_2d'].values, df_test['Flood'])
for k, v in accuracy_prod.items():
    print((k), ': ', round(v,4))

Forecast Accuracy of: Min Temperature
mape :  3.4519
me :  -1.1881
mae :  7.504
mpe :  -0.2386
rmse :  8.5891
corr :  0.073
minmax :  0.4736

Forecast Accuracy of: Max Temperature
mape :  0.7281
me :  -0.6049
mae :  6.8663
mpe :  0.0836
rmse :  8.2356
corr :  0.0473
minmax :  0.2416

Forecast Accuracy of: Precipitation
mape :  inf
me :  -0.4764
mae :  6.4918
mpe :  nan
rmse :  12.4681
corr :  0.0129
minmax :  inf

Forecast Accuracy of: Min_Rel Humidity
mape :  0.2872
me :  -0.1426
mae :  10.8784
mpe :  0.1102
rmse :  13.9184
corr :  0.0175
minmax :  0.2094

Forecast Accuracy of: Max_Rel Humidity
mape :  0.1145
me :  3.0222
mae :  8.8944
mpe :  0.0554
rmse :  11.8265
corr :  -0.0078
minmax :  0.0937

Forecast Accuracy of: Wind Speed
mape :  0.3655
me :  -0.2088
mae :  1.1746
mpe :  0.1179
rmse :  1.5136
corr :  0.0455
minmax :  0.2627

Forecast Accuracy of: Vapor Pressure Deficit
mape :  inf
me :  -0.0678
mae :  0.3939
mpe :  inf
rmse :  0.4971
corr :  0.0214
minmax :  0.3432

Forecast 

<ipython-input-17-6c745c477bee>:10: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  actual[:,None]]), axis=1)
<ipython-input-17-6c745c477bee>:12: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  actual[:,None]]), axis=1)
<ipython-input-17-6c745c477bee>:13: RuntimeWarning: divide by zero encountered in true_divide
  minmax = 1 - np.mean(mins/maxs)             # minmax
